In [2]:
!pip install soundfile 

     ---------------------------------------- 1.0/1.0 MB 9.2 MB/s eta 0:00:00


In [6]:
import sounddevice as sd
import soundfile as sf
import tkinter as tk
from threading import Thread
import queue
import datetime

# Globale Variablen
SAMPLING_RATE = 44100  # Sampling rate in Hz
DURATION = 1.1  # Dauer der Aufnahme in Sekunden
audio_queue = queue.Queue()

def record_audio(key_name):
    # 0.1 Sekunden Vorlauf + 1 Sekunde Aufnahme
    myrecording = sd.rec(int(DURATION * SAMPLING_RATE), samplerate=SAMPLING_RATE, channels=2)
    sd.wait()  # Warten, bis die Aufnahme abgeschlossen ist
    audio_queue.put((myrecording, key_name))

def on_key_press(event):
    # Starte die Aufnahme in einem separaten Thread
    key_name = event.keysym  # Name der gedrückten Taste
    record_thread = Thread(target=record_audio, args=(key_name,))
    record_thread.start()

    # Speichere die Audiodatei, wenn die Aufnahme beendet ist
    record_thread.join()
    recording, key_name = audio_queue.get()
    filename = f"{key_name}_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.wav"
    sf.write(filename, recording, SAMPLING_RATE)
    print(f"Audio saved as {filename}")

# Erstellen des Tkinter-Fensters
root = tk.Tk()
root.title("Audio Recorder")

# Globalen Tastendruck-Ereignishandler einrichten
root.bind("<KeyPress>", on_key_press)

# Starte das Tkinter-Hauptfenster
root.mainloop()


Audio saved as 1_2024-01-13_15-14-41.wav
Audio saved as 2_2024-01-13_15-14-45.wav
Audio saved as 2_2024-01-13_15-14-48.wav
Audio saved as 3_2024-01-13_15-14-50.wav
Audio saved as 4_2024-01-13_15-14-52.wav
Audio saved as a_2024-01-13_15-14-55.wav
Audio saved as b_2024-01-13_15-14-57.wav
Audio saved as c_2024-01-13_15-14-59.wav
Audio saved as d_2024-01-13_15-15-01.wav


In [9]:
import sounddevice as sd
import numpy as np
import queue
import threading

# Globale Variablen
SAMPLING_RATE = 44100
BUFFER_DURATION = 1.1  # 0.1 Sekunden vor und 1 Sekunde nach dem Drücken
buffer_size = int(BUFFER_DURATION * SAMPLING_RATE)
audio_buffer = np.zeros((buffer_size, 2), dtype=np.float32)
audio_queue = queue.Queue()

# Funktion, um kontinuierlich Audio aufzunehmen
def audio_callback(indata, frames, time, status):
    global audio_buffer
    audio_buffer = np.roll(audio_buffer, -frames, axis=0)
    audio_buffer[-frames:, :] = indata

# Starten der kontinuierlichen Aufnahme
stream = sd.InputStream(callback=audio_callback, channels=2, samplerate=SAMPLING_RATE)
stream.start()

# Funktion, um Audio bei Tastendruck aufzunehmen
def record_on_keypress():
    while True:
        key = input("Drücken Sie eine Taste, um aufzunehmen (q zum Beenden): ")
        if key.lower() == 'q':
            break
        current_buffer = np.copy(audio_buffer)
        audio_queue.put(current_buffer)

# Starten des Threads für die Tastenaufnahme
keypress_thread = threading.Thread(target=record_on_keypress)
keypress_thread.start()


Drücken Sie eine Taste, um aufzunehmen (q zum Beenden): 

In [1]:
import sounddevice as sd
import numpy as np
import queue
import threading
import soundfile as sf
import datetime
from pynput import keyboard

# Globale Variablen
SAMPLING_RATE = 44100
BUFFER_DURATION = 1.1  # 0.1 Sekunden vor und 1 Sekunde nach dem Drücken
buffer_size = int(BUFFER_DURATION * SAMPLING_RATE)
audio_buffer = np.zeros((buffer_size, 2), dtype=np.float32)
audio_queue = queue.Queue()

# Funktion, um kontinuierlich Audio aufzunehmen
def audio_callback(indata, frames, time, status):
    global audio_buffer
    audio_buffer = np.roll(audio_buffer, -frames, axis=0)
    audio_buffer[-frames:, :] = indata

# Starten der kontinuierlichen Aufnahme
stream = sd.InputStream(callback=audio_callback, channels=2, samplerate=SAMPLING_RATE)
stream.start()

# Funktion zum Speichern der Audiodaten
def save_audio():
    while True:
        item = audio_queue.get()
        if item is None:
            break  # Beenden, wenn None als Signal zum Beenden empfangen wird
        audio_data, key_name = item
        filename = f"./Data/{key_name}_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.wav"
        sf.write(filename, audio_data, SAMPLING_RATE)
        print(f"Audio saved as {filename}")

# Funktion, um auf Tastendruck zu reagieren
def on_press(key):
    try:
        key_name = key.char  # Versuche, das Zeichen der Taste zu erhalten
    except AttributeError:
        key_name = key.name  # Wenn es kein Zeichen gibt, verwende den Namen der Taste

    current_buffer = np.copy(audio_buffer)
    audio_queue.put((current_buffer, key_name))

    if key == keyboard.Key.esc:
        # Beenden, wenn die Escape-Taste gedrückt wird
        return False

# Listener für Tastatureingaben
listener = keyboard.Listener(on_press=on_press)
listener.start()

# Starten des Speicher-Threads
save_thread = threading.Thread(target=save_audio)
save_thread.start()

# Warte, bis der Listener beendet wird (durch Drücken der Escape-Taste)
listener.join()

# Signal zum Beenden an den Speicher-Thread senden und warten
audio_queue.put(None)
save_thread.join()

ModuleNotFoundError: No module named 'pynput'

In [2]:
!pip install pynput


     ---------------------------------------- 89.2/89.2 kB 2.5 MB/s eta 0:00:00
